In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl

In [ ]:
df = pd.read_excel('Magnetics-Curve-Fit-Equation-Tool-2020.xlsx', sheet_name="Magnetics Curve Fit Equations", skiprows=7)

In [ ]:
mpl.rcParams["figure.figsize"] = 10,8
mpl.rcParams["axes.grid"] = True

In [ ]:
def calc_rel_perm_ifo_dcb_oersted(oersted, factors):
    return (1/(factors["a"]+factors["b"]*(oersted**factors["c"])))
def calc_core_loss_density(flux_density, sw_freq, factors):
    return (factors["a"] * (flux_density**factors["b"]) * (sw_freq**factors["c"]))
def calc_magnetization(oersted, factors):
    return ((factors["a"] + factors["b"]*oersted + factors["c"]*oersted**2) / (1 + factors["d"]*oersted + factors["e"]*oersted**2))**factors["x"]
def match_material_perm(df, mat_col, perm_col, mat, perm):
    filtered = df[(df[mat_col] == mat) & (df[perm_col] == perm)] # AND logic for two
    return filtered.drop([mat_col, perm_col], axis=1)
def create_df_from_named_cols(df, cols, threshold, rename_dict={}):
    dropped = df.dropna(axis=1, how="all")
    return dropped[cols].dropna(axis=0, thresh=threshold).fillna(method="ffill").rename(columns=rename_dict)

In [ ]:
perm_vs_dc_bias = create_df_from_named_cols(df, ["Material:", "Initial Permeability", "a", "b", "c"], 3, rename_dict={"Material:": "Material"})
rename_core_loss = {"Material:.1": "Material", "Initial Permeability.1": "Initial Permeability", "a.1": "a", "b.1": "b", "c.1": "c"}
core_loss_dens_vs_flux_dens_sw_freq = create_df_from_named_cols(df, ["Material:.1", "Initial Permeability.1", "a.1", "b.1", "c.1"], 3, rename_dict=rename_core_loss)
rename_magnetization = {"a.2": "a", "b.2": "b", "c.2": "c"}
magnetization = create_df_from_named_cols(df, ["Material", "Permeability", "a.2", "b.2", "c.2", "d", "e", "x"], 5, rename_dict=rename_magnetization)

In [ ]:
oer = list(range(100))
factors = match_material_perm(perm_vs_dc_bias, "Material", "Initial Permeability", "MPP", 160)
factors2 = match_material_perm(perm_vs_dc_bias, "Material", "Initial Permeability", "MPP", 200)
plt.plot(oer, [calc_rel_perm_ifo_dcb_oersted(x, factors) for x in oer])
plt.plot(oer, [calc_rel_perm_ifo_dcb_oersted(x, factors2) for x in oer])
plt.show(block = True)
perm_vs_dc_bias.to_pickle("mu_perc_vs_H.pkl")

In [ ]:
flux_dens = list(map(lambda x: x/500, range(100)))
print(flux_dens)
sw_freq = 100
factors = match_material_perm(core_loss_dens_vs_flux_dens_sw_freq, "Material", "Initial Permeability", "MPP", 160)
factors2 = match_material_perm(core_loss_dens_vs_flux_dens_sw_freq, "Material", "Initial Permeability", "MPP", 200)
plt.plot(flux_dens, [calc_core_loss_density(x, sw_freq, factors) for x in flux_dens])
plt.plot(flux_dens, [calc_core_loss_density(x, sw_freq, factors2) for x in flux_dens])
plt.show(block = True)
core_loss_dens_vs_flux_dens_sw_freq.to_pickle("Pc_vs_B-fsw.pkl")

In [ ]:
oer = list(range(100))
factors = match_material_perm(magnetization, "Material", "Permeability", "MPP", 160)
factors2 = match_material_perm(magnetization, "Material", "Permeability", "MPP", 200)
plt.plot(oer, [calc_magnetization(x, factors) for x in oer])
plt.plot(oer, [calc_magnetization(x, factors2) for x in oer])
plt.show(block = True)
magnetization.to_pickle("B_vs_H.pkl")